In [41]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-04-23 12:09:07--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8000::154, 2606:50c0:8001::154, 2606:50c0:8002::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8000::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.2’

input.txt.2         100%[===================>]   1.06M   980KB/s    in 1.1s    

2023-04-23 12:09:08 (980 KB/s) - ‘input.txt.2’ saved [1115394/1115394]



In [1]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [2]:
len(text)

1115394

In [3]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [4]:
chars = sorted(list(set(text)))
vocab_size = len(chars)

In [5]:
print(''.join(chars))
vocab_size


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


65

In [6]:
stoi = { ch:i for i, ch in enumerate(chars) }
itos = {i:ch for i,ch in enumerate(chars)}

In [7]:
itos = {i:ch for i,ch in enumerate(chars)}

In [8]:
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: "".join([itos[x] for x in l])

In [9]:
decode = lambda l: "".join([itos[x] for x in l])

In [10]:
encode("hi there")

[46, 47, 1, 58, 46, 43, 56, 43]

In [11]:
decode([46, 47, 1, 58, 46, 43, 56, 43])

'hi there'

In [12]:
import torch

In [13]:
data = torch.tensor(encode(text), dtype = torch.long)

In [14]:
data.shape, type(data)

(torch.Size([1115394]), torch.Tensor)

In [15]:
data[:1000]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
        47, 59, 57,  1, 47, 57,  1, 41, 

In [16]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [17]:
train_data = data[:n]
val_data = data[n:]

In [18]:
val_data = data[n:]

In [19]:
block_size = 8

In [20]:
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [21]:
x = train_data[:block_size]

In [22]:
y = train_data[1:block_size+1]

In [23]:
x,y

(tensor([18, 47, 56, 57, 58,  1, 15, 47]),
 tensor([47, 56, 57, 58,  1, 15, 47, 58]))

In [24]:
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print("ctx ", context, "target", target)

ctx  tensor([18]) target tensor(47)
ctx  tensor([18, 47]) target tensor(56)
ctx  tensor([18, 47, 56]) target tensor(57)
ctx  tensor([18, 47, 56, 57]) target tensor(58)
ctx  tensor([18, 47, 56, 57, 58]) target tensor(1)
ctx  tensor([18, 47, 56, 57, 58,  1]) target tensor(15)
ctx  tensor([18, 47, 56, 57, 58,  1, 15]) target tensor(47)
ctx  tensor([18, 47, 56, 57, 58,  1, 15, 47]) target tensor(58)


In [36]:
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1137)

In [37]:
batch_size = 4

In [38]:
block_size = 8

In [39]:
torch.randint(6, (4,))

tensor([0, 5, 3, 1])

In [40]:
# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

In [41]:
xb, yb = get_batch('train')

In [42]:
for b in range(batch_size):
    for t in range(block_size):
        context = xb[b][:t+1]
        target = yb[b][t]
        print(context, "-----", target)

tensor([1]) ----- tensor(58)
tensor([ 1, 58]) ----- tensor(46)
tensor([ 1, 58, 46]) ----- tensor(43)
tensor([ 1, 58, 46, 43]) ----- tensor(1)
tensor([ 1, 58, 46, 43,  1]) ----- tensor(46)
tensor([ 1, 58, 46, 43,  1, 46]) ----- tensor(53)
tensor([ 1, 58, 46, 43,  1, 46, 53]) ----- tensor(54)
tensor([ 1, 58, 46, 43,  1, 46, 53, 54]) ----- tensor(43)
tensor([53]) ----- tensor(1)
tensor([53,  1]) ----- tensor(39)
tensor([53,  1, 39]) ----- tensor(1)
tensor([53,  1, 39,  1]) ----- tensor(61)
tensor([53,  1, 39,  1, 61]) ----- tensor(47)
tensor([53,  1, 39,  1, 61, 47]) ----- tensor(44)
tensor([53,  1, 39,  1, 61, 47, 44]) ----- tensor(43)
tensor([53,  1, 39,  1, 61, 47, 44, 43]) ----- tensor(0)
tensor([39]) ----- tensor(52)
tensor([39, 52]) ----- tensor(42)
tensor([39, 52, 42]) ----- tensor(56)
tensor([39, 52, 42, 56]) ----- tensor(63)
tensor([39, 52, 42, 56, 63]) ----- tensor(1)
tensor([39, 52, 42, 56, 63,  1]) ----- tensor(44)
tensor([39, 52, 42, 56, 63,  1, 44]) ----- tensor(53)
tensor([

In [43]:
vocab_size

65

In [44]:

def get_decay_matrix(dim, gamma):
    d = torch.ones(dim)
    d = torch.tril(d)
    
    for index, head in enumerate(d):
        g = gamma[index]
        for idx, x in enumerate(torch.tril(head)):
            for idy, y in enumerate(x):
                if idx >= idy:
                    head[idx][idy] = g ** (idx-idy)
    return d

In [45]:
import einops
from einops import rearrange, reduce, repeat


In [53]:

class ChunkwiseRetention(nn.Module):
    def __init__(self, chunk_size, num_head, block_size):
        super().__init__()
        self.key = nn.Linear(n_embed,  chunk_size * num_head, bias = False)
        self.query = nn.Linear(n_embed,  chunk_size * num_head, bias = False)
        self.value = nn.Linear(n_embed,  chunk_size * num_head, bias = False)
        self.gamma = 1.0-2.0**(-5-torch.arange(0,num_head))
        self.decay_mask = get_decay_matrix((num_head, block_size, block_size), self.gamma)
        self.chunk_decay = self.gamma
        self.gn = nn.GroupNorm(1, num_head)
        self.num_head = num_head
        self.chunk_size = chunk_size
        
    def forward(self, x, past_kv):
        
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        v = self.value(x)
        
        k = rearrange(k, ('b t (h c) -> b h t c'), t=T, h=self.num_head, c =self.chunk_size)
        q = rearrange(q, ('b t (h c) -> b h t c'), t=T, h=self.num_head, c =self.chunk_size)
        v = rearrange(v, ('b t (h c) -> b h t c'), t=T, h=self.num_head, c =self.chunk_size)
        
    
        retention = q @ k.transpose(-1, -2)
        
        # b h t c , b h c t -> b h t t 
    
        retention = retention  * self.decay_mask   # b h t t* h t t
        
        
        
        inner_retention = retention @ v
        
        
        past_kv = repeat(past_kv, 'n q v -> B n q v', B=B)
        pb, pn, pq, pv = past_kv.shape
        
        padding = torch.zeros(pb, pn, pq, self.chunk_size)
        
        past_kv = past_kv+ padding
        
        
        
        dm = repeat(self.decay_mask, 'h c d -> B h c d', B=B)
        pp = q @ past_kv
        cross_retention = pp.transpose(-1, -2) @ dm
        cross_retention = cross_retention.transpose(-1, -2)
        
        
        retention = inner_retention + cross_retention
        
        
        
        current_kv = self.gamma.view(self.num_head, 1, 1) * past_kv + (k.transpose(-1, -2) @ v)
        output = self.gn(retention.transpose(-1,-2))
        output = rearrange(output, 'b c h t -> b t (c h)')
        return output, current_kv.mean(dim=0)
    
    
class GatedMultiScaleRetention(nn.Module):
    def __init__(self, chunk_size, num_head, block_size):
        super().__init__()
        self.wg = nn.Linear(n_embed,  n_embed, bias = False)
        self.act = nn.SiLU()
        self.y= ChunkwiseRetention(num_head = n_head, chunk_size = n_embed//n_head, block_size=block_size)
        self.wo = nn.Linear(n_embed,  n_embed, bias = False)
        self.past = torch.zeros(num_head, chunk_size, chunk_size)
        
    def forward(self, x):
        wgx = self.wg(x)
        wgx = self.act(wgx)
        y, past = self.y(wgx, self.past)
        self.past = past.detach()
        y = wgx * y
        return self.wo(y)
        

class FeedForward(nn.Module):
    def __init__(self, n_embed):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embed, 4* n_embed),
            nn.GELU(),
            nn.Linear(4 * n_embed, n_embed),
         nn.Dropout(dropout))
    
    def forward(self, x):
        return self.net(x)
    
class Block(nn.Module):
    def __init__(self, n_embed, n_head, block_size):
        super().__init__()
        self.sa_head= GatedMultiScaleRetention(num_head = n_head, chunk_size = n_embed//n_head, block_size=block_size)
        self.ffw=  FeedForward(n_embed)
        self.ln1 = nn.LayerNorm(n_embed)
        self.ln2 = nn.LayerNorm(n_embed)
    
    def forward(self, x):
        x = x + self.sa_head(self.ln1(x))
        x = x+self.ffw(self.ln2(x))
        return x
        

class RetNet(nn.Module):
    def __init__(self, block_size):
        super().__init__()
        
        self.token_embedding_table = nn.Embedding(vocab_size, n_embed)
        self.position_embedding_table = nn.Embedding(block_size, n_embed)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head, block_size=block_size) for _ in range(n_layer)])
        self.lm_head = nn.Linear(n_embed, vocab_size)

        
    def forward(self, idx, targets=None):
        B, T = idx.shape
        
        token_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T))
        x = token_emb + pos_emb
        x = self.blocks(x)
        logits = self.lm_head(x)
        if targets == None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    
    def generate(self, idx, max_new_tokes):
        for _ in range(max_new_tokes):
            b, s = idx.shape
            bk = min(s, block_size)
            idx_cond =  torch.cat((torch.zeros(b, block_size-bk, dtype=int), idx), dim=1)[:, -block_size:]
            logits, loss = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim = -1)
            idx_next = torch.multinomial(probs, num_samples = 1)
            idx = torch.cat((idx, idx_next), dim = 1)
        return idx

In [54]:
# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_head = 4
n_layer = 4
dropout = 0.0
n_embed = 32
# ------------

In [55]:
# data loading
def get_batch(split, batch_size):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

In [56]:
model = RetNet(block_size=block_size)
xb, yb = get_batch('train', batch_size=batch_size)
optimizer = torch.optim.AdamW(model.parameters(),lr=learning_rate)
logits, loss = model(xb, yb)

In [912]:
loss.shape

torch.Size([])

In [62]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split, batch_size=batch_size)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [63]:
xb, yb = get_batch('train', batch_size=batch_size)

In [64]:
logits, loss = model(xb, yb)

In [65]:

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % 100 == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train', batch_size=batch_size)

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 4.5199, val loss 4.5151
step 100: train loss 2.8002, val loss 2.8240
step 200: train loss 2.6565, val loss 2.7735
step 300: train loss 2.6466, val loss 2.8999
step 400: train loss 2.8889, val loss 3.2265
step 500: train loss 2.4477, val loss 2.6071
step 600: train loss 2.5639, val loss 2.8179
step 700: train loss 2.3692, val loss 2.6647
step 800: train loss 2.9142, val loss 3.4041
step 900: train loss 2.2591, val loss 2.3663
step 1000: train loss 2.5290, val loss 2.9728
step 1100: train loss 2.2890, val loss 2.3944
step 1200: train loss 2.7051, val loss 3.1527
step 1300: train loss 2.5241, val loss 2.8266
step 1400: train loss 2.3235, val loss 2.5941
step 1500: train loss 2.5920, val loss 3.0807
step 1600: train loss 2.5082, val loss 2.8587
step 1700: train loss 2.6868, val loss 2.9852
step 1800: train loss 2.6550, val loss 3.3294
step 1900: train loss 3.2855, val loss 3.4901
step 2000: train loss 2.6440, val loss 3.1719
step 2100: train loss 2.8744, val loss 3.6023


In [71]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(model.generate(context, max_new_tokes=200)[0].tolist()))


HUThioth, faledisw brohid hais anow gioun'nthaseriloay mo; thavehen LAwar bobul, traga mironthaltith, oul tha be mtou; so.
SThald CORThen ns hwid boucoche f gwitifen l?
I titin ild itsed p geanovAnh s


In [69]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(model.generate(context, max_new_tokes=200)[0].tolist()))


I tacoghedinofo fstridil iraing o, tmitio itsoshene f:
Astetne hai on o by ardin urdit Ci sp teate d d oigdost s HAnont?
ICantod homs coke ayer ENZtousu.
In'Thitaidorncosy f venely nou-QY s dEy.
O:
SC


In [72]:
import tiktoken
enc = tiktoken.get_encoding("cl100k_base")
assert enc.decode(enc.encode("hello world")) == "hello world"

# To get the tokeniser corresponding to a specific model in the OpenAI API:
enc = tiktoken.encoding_for_model("gpt-4")

In [73]:
assert enc.decode(enc.encode("hello world")) == "hello world"


In [74]:
enc.encode("hello world")

[15339, 1917]

In [75]:
text_tokens = enc.encode(text)

In [76]:
len(text_tokens)

301829

In [77]:
enc.n_vocab

100277

In [78]:
enc.decode([text_tokens[0]])

'First'

In [79]:
data = torch.tensor(text_tokens, dtype = torch.long)

In [80]:
data.shape

torch.Size([301829])

In [83]:
learning_rate = 3e-4

In [84]:
len(text.split(' '))

169893

In [85]:
len(text)

1115394

In [86]:
chars = sorted(list(set(text.split(' '))))
vocab_size = len(chars)

In [87]:
chars[2]

'\nWas'

In [88]:
stoi = { ch:i for i, ch in enumerate(chars) }
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: " ".join([itos[x] for x in l])

In [89]:
data = torch.tensor(encode(text.split(' ')), dtype = torch.long)

In [90]:
data[:10]

tensor([ 1455,   957, 39874, 29614,  5949, 16628, 18572, 24432, 34050, 34057])

In [91]:
decode(encode(text.split(' ')[:10]))

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst'

In [92]:
text.split(' ')[:10]

['First',
 'Citizen:\nBefore',
 'we',
 'proceed',
 'any',
 'further,',
 'hear',
 'me',
 'speak.\n\nAll:\nSpeak,',
 'speak.\n\nFirst']

In [93]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [95]:
# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 32
n_embed = 32
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

In [96]:
model = RetNet(block_size=block_size)
xb, yb = get_batch('train', batch_size=batch_size)
optimizer = torch.optim.AdamW(model.parameters(),lr=learning_rate)
logits, loss = model(xb, yb)

In [109]:

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % 100 == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train', batch_size=batch_size)

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 7.2402, val loss 9.8991
step 100: train loss 6.9271, val loss 10.1411
step 200: train loss 6.2498, val loss 9.3603
step 300: train loss 6.1858, val loss 9.5330
step 400: train loss 6.1536, val loss 9.6789
step 500: train loss 6.0791, val loss 9.7732
step 600: train loss 6.9841, val loss 10.4825
step 700: train loss 6.0675, val loss 9.9038
step 800: train loss 6.2187, val loss 10.2573
step 900: train loss 6.2173, val loss 10.2595
step 1000: train loss 5.8269, val loss 10.1152
step 1100: train loss 6.1537, val loss 10.7570
step 1200: train loss 5.7523, val loss 10.2635
step 1300: train loss 6.0812, val loss 10.6564
step 1400: train loss 5.6627, val loss 10.4134
step 1500: train loss 5.7125, val loss 10.4211
step 1600: train loss 5.4483, val loss 10.4002
step 1700: train loss 6.1413, val loss 11.2838
step 1800: train loss 5.9623, val loss 11.1319
step 1900: train loss 5.5093, val loss 10.5941
step 2000: train loss 5.3935, val loss 10.6224
step 2100: train loss 5.5008, v

In [110]:
context = torch.tensor([encode("thou art kneel before king".split(' '))], dtype = torch.long)
print(decode(model.generate(context, max_new_tokes=200)[0].tolist()))


thou art kneel before king this holy friar, now, triumph like to the flood?
Yet bands before what we cheque to their words and now the apostle vengeance to the market-place!
You will would
When husband lurch'd a father from the comfort so dishonour'd me hear this business straight;
Go, more fair time you'll make, as e'er that fear'd where suspicion from my lord, I'll buy justice, justice!

DUKE VINCENTIO:
Relate hopeless restitution, I'll depose thrice fond heaven
Boldly delight to the most kindred unto Bianca
Till Katharina blame if the moiety. thou revenged the contrary gives young Rutland, prosecute
'Gainst quench his ears, for what's a case of you.

ISABELLA:
Good Come, the lords who stood Susan before.

KING as an
inventory and one friend.

KING man of my day,
That service for the fume O, scars and, of him;
Bear my body's length.
Why, 'tis charity.
What, return.

HENRY BOLINGBROKE:
Nay, for my remission copy of the greatest her unto mine own noble lord.

BRAKENBURY:
Why expected.


In [111]:
context = torch.tensor([encode("thou art kneel before king".split(' '))], dtype = torch.long)
print(decode(model.generate(context, max_new_tokes=200)[0].tolist()))


thou art kneel before king he that her no, such delicate
burthens shall stay with rice? man said that hast as wise For Come, Escalus,
You will I.

DUCHESS i' the king till my upon! you'll continue so bad lord, you in twain.

CLAUDIO:
But that's mine own noble ladies,--and all to me!
God for them than a pretty Camillo,--
But friar, denies prudence; city gates,
Speak at bowls.

QUEEN:
'Twill had woe! with him.
Off curses may live:
There alack the world wise say'st thou hast thou sworest that drops of their idle body,
But to unwise you
To will serve
For you in the husband or is my peril in virtue,
I as your wisdom be
not tainted! even that you some Paris; as I will all acknowledge. Lord Hastings, which I should name of his own power: you to we
shall let us as you,
A nuns, upon my master time against his summer butterflies,
Or butchers 'Romeo who, the spices and say where he hears even drag no healthsome man was been beaten true sign is he? I well.

ANGELO:
Good father, is, or woe.

Nurse:

In [113]:
context = torch.tensor([encode("Hermione".split(' '))], dtype = torch.long)
print(decode(model.generate(context, max_new_tokes=200)[0].tolist()))


Hermione in God's enemy,
God is having already mine majesty may remain that afford any fear
Lesser him brazen pale to the other of Capulet's
Sups me with age, ache, cause and not queen upon his treasure is:
He do know half you hour mightst have but forth Jesu money, with all slaughter'd lands.

LADY GREY:
Be reform'd: that is the news?

COMINIUS:
Your law repair of my lord; our security,
Grows wife, that can wit but to the fault of my neighbour youngest promises a bear.

MENENIUS:
He's thy sort.

RICHARD:
'Twas be crowned forty years,
My wretch,
Grace for all my gracious lord:
I dream till since thee Caesar never grow.

GARDENER:
Poor so? so, nor vantage I'll not whom the duke and tie bloody cannibals!
How your cousin?

JULIET:
Shall I am glad RICHARD II:
Join your hand,
And know you may.
Say, down, sure, Tybalt, the worse:
Fell love, many shepherd is importune his rest,
That a day and let us here longer.

LEONTES:
Ha!

CAMILLO:
Stays toil, the lieutenant comes.
Master bitterly:
'Yea,'

In [114]:
context = torch.tensor([encode("come".split(' '))], dtype = torch.long)
print(decode(model.generate(context, max_new_tokes=200)[0].tolist()))


come mind my son of my country's comfort on an ounce.

AUTOLYCUS:

Shepherd:
Well, with fire
Our ever your great guess, as easy
As sin!
Shall 'tis an unpitied end:
Earth waged her suit;
It whose tongue beat thee here. hands: my fierce good, his princely semblance
Are of my head to me,
And nor your way.

VOLUMNIA:
O, man;
With his discords at the noon-tide night.
Princes be loud am a thief to avoid,
And by blood and wear favour do some MARGARET:
Thy woes desperately never man
Sigh'd pardon to hear to sweetly any you in pieces toward men, his assistants, as with fantastic not hurt not much amiss: mother should miss off a dangerous words bewitch me tell me,
In gentle Percy; as is no other, or mark you
His legs and speak any scruple; your gate
And therein,
And shall advise you
To sit She sir, on Warwick, tongue in now.

PETER:
You cannot lose their dusky graves.
Richard fair for some bond, the seat, you both I cannot more than the hire
Of 'gainst the dark.

MERCUTIO:
If wounded forth; pain